In [ ]:
import pandas as pd
from phoenix.db import models
from sqlalchemy import and_, create_engine, select
from sqlalchemy.orm import aliased, sessionmaker

In [ ]:
# PostgresSession = sessionmaker(
#     create_engine(
#         "postgresql+psycopg://localhost:5432/postgres?user=postgres&password=mysecretpassword",
#         echo=True,
#     ),
#     expire_on_commit=False,
# )
SqliteSession = sessionmaker(
    create_engine("sqlite:////Users/xandersong/.phoenix/phoenix.db", echo=True),
    expire_on_commit=False,
)

In [ ]:
orig_endpoint = "http://127.0.0.1:6008"
postgres_endpoint = "http://127.0.0.1:6007"
sqlite_endpoint = "http://127.0.0.1:6006"

In [ ]:
# SELECT * FROM spans
# JOIN (
#     SELECT spans.id, sa.score, sa.label FROM spans
#     JOIN span_annotations sa on spans.id = sa.span_rowid
# ) B ON spans.id == B.id
# WHERE B.score > 0.5 AND B.label == 'factual' AND spans.name == 'query';

In [ ]:
with SqliteSession() as session:
    df = pd.read_sql(
        select(models.Span.id)
        .join(models.SpanAnnotation)
        .where(models.SpanAnnotation.score > 0.5)
        .where(models.SpanAnnotation.label == "factual"),
        session.connection(),
    )
df

```
evals["Q&A Correctness"].label == "correct" and evals["Hallucination"].label == "hallucinated"
```

In [ ]:
with SqliteSession() as session:
    A = aliased(models.SpanAnnotation)
    B = aliased(models.SpanAnnotation)
    df = pd.read_sql(
        select(models.Span.id, A.name, A.label, B.name, B.label)
        .join(A)
        .join(B)
        .where(
            and_(
                A.name == "Q&A Correctness",
                A.label == "correct",
                B.name == "Hallucination",
                B.label == "hallucinated",
            ),
        ),
        session.connection(),
    )
df

```
evals["Q&A Correctness"].label == "correct" and evals["Hallucination"].score == 0
```

In [ ]:
with SqliteSession() as session:
    A = aliased(models.SpanAnnotation)
    B = aliased(models.SpanAnnotation)
    df = pd.read_sql(
        select(models.Span.id, A.name, A.label, B.name, B.label)
        .join(A)
        .join(B)
        .where(
            and_(
                A.name == "Q&A Correctness",
                A.label == "correct",
                B.name == "Hallucination",
                B.score == 0,
            ),
        ),
        session.connection(),
    )
df

In [ ]:
with SqliteSession() as session:
    A = aliased(models.SpanAnnotation)
    df = pd.read_sql(
        select(models.Span.id, A.name, A.label)
        .join(A)
        .where(
            and_(
                A.name == "Q&A Correctness",
                A.label == "correct",
            ),
        ),
        session.connection(),
    )
df

In [ ]:
# SELECT span_annotations.span_rowid,
# MAX(CASE WHEN name = 'Hallucination' and score = 0 THEN 1 ELSE 0 END) AS A,
# MAX(CASE WHEN name = 'Q&A Correctness' and label = 'correct' THEN 1 ELSE 0 END) AS B
# FROM span_annotations
# WHERE name in ('Hallucination', 'Q&A Correctness')
# GROUP BY span_annotations.span_rowid
# HAVING A = 1 and B = 1
# ORDER BY span_rowid;